<a href="https://colab.research.google.com/github/a-forty-two/cog_may_onwards_21/blob/main/Cog_cv_tf_ObjectDetection_finale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Users provide us inputs(X), and outputs (Y)

but in order to calculate-> M and C-> i have a lot of other parameters

1) Learning Rate

2) Epochs

3) L1/L2

4) intiatial values.... 


VALUES that you need, but are NOT provided by the SME/data providers

SO we try to play around with the values-> to figure the best answer

1) Brute force-> all possible combinations, and the best combination is selected. GRID SEARCH.

2) Compromised on efficiency/Optimized on time-> INFINITE possible answers, so generate humongous amount of random pairs, and then select best pair out of them. So we get highly workable parameters, not the best one. -> Randomized Search

3) Probability: Next guess is selected based on current performance of HP combination-> Bayesian method

In [5]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras import backend as k
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

In [6]:
HP_filter1 = (3,3)
HP_filter2 = (2,2)
HP_epochs = 20
HP_init_lr = 0.01
HP_image_dim = (96,96,3)
HP_batch_size = 1

In [7]:
class AnacondaVGG:
  @staticmethod
  def buildmodel(height, width, depth, classes):
    channel_dim = -1
    #block1
    inputshape = (height, width, depth)
    model = keras.Sequential()
    model.add(Conv2D(32, HP_filter1, padding='same', input_shape=inputshape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_filter1))
    model.add(Dropout(0.20))

    #block 2
    model.add(Conv2D(64, HP_filter1, padding='same'))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(64, HP_filter1, padding='same'))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_filter2))
    model.add(Dropout(0.20))

    #block 3
    model.add(Conv2D(128, HP_filter1, padding='same'))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(128, HP_filter1, padding='same'))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_filter2))
    model.add(Dropout(0.20))

    #block 4
    model.add(Conv2D(256, HP_filter1, padding='same'))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(256, HP_filter1, padding='same'))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_filter2))
    model.add(Dropout(0.20))



    #block_final
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dense(classes))
    model.add(Activation("softmax"))

    return model

In [8]:
dataset = 'data'
modelpath = 'model'
outputpath = 'model/labels.bin'
metricspath = 'model/metrics'
evaldataset = ''
testdataset = ''

In [9]:
import imutils
from imutils import paths
allimages = sorted(list(paths.list_images(dataset)))

In [10]:
allimages

['data/cylons/00000000.jpeg',
 'data/cylons/00000001.jpeg',
 'data/cylons/00000002.jpeg',
 'data/cylons/00000003.jpeg',
 'data/cylons/00000004.jpeg',
 'data/cylons/00000005.jpeg',
 'data/cylons/00000006.jpeg',
 'data/cylons/00000007.jpeg',
 'data/cylons/00000008.jpeg',
 'data/cylons/00000009.jpeg',
 'data/cylons/00000010.jpeg',
 'data/cylons/00000011.jpeg',
 'data/pistol/00000000.jpeg',
 'data/pistol/00000001.jpeg',
 'data/pistol/00000002.jpeg',
 'data/pistol/00000003.jpeg',
 'data/pistol/00000004.jpeg',
 'data/pistol/00000005.jpeg',
 'data/pistol/00000006.jpeg',
 'data/pistol/00000007.jpeg',
 'data/pistol/00000008.jpeg',
 'data/pistol/00000009.jpeg',
 'data/pistol/00000010.jpeg',
 'data/pistol/00000011.jpeg']

In [11]:
import random
random.seed(42)
random.shuffle(allimages)

In [12]:
allimages[:5]

['data/pistol/00000003.jpeg',
 'data/cylons/00000005.jpeg',
 'data/pistol/00000006.jpeg',
 'data/pistol/00000011.jpeg',
 'data/pistol/00000001.jpeg']

In [13]:
import os
import cv2
from keras.preprocessing.image import img_to_array
data = []
labels = []
for impath in allimages:
  img = cv2.imread(impath)
  resized = cv2.resize(img, (96,96))
  imgdata = img_to_array(resized)
  data.append(imgdata)
  label = impath.split(os.path.sep)[-2]
  labels.append(label)

In [14]:
data[0][0][0] # FIRST pixel, of first row of first unit in dataset

array([26., 33., 37.], dtype=float32)

In [15]:
print(labels[:10])

['pistol', 'cylons', 'pistol', 'pistol', 'pistol', 'cylons', 'pistol', 'cylons', 'pistol', 'pistol']


In [16]:
from sklearn.preprocessing import LabelBinarizer
labels = np.array(labels)
lb = LabelBinarizer()
binarizedlabels = lb.fit_transform(labels)
print(binarizedlabels[:5])

[[1]
 [0]
 [1]
 [1]
 [1]]


In [17]:
normdata = np.array( data, dtype='float')/255

In [18]:
from sklearn.model_selection import train_test_split
trainx, testx, trainy, testy = train_test_split(normdata, binarizedlabels, test_size=0.2, random_state=42)

In [19]:
from keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2,
                         zoom_range=0.2, horizontal_flip=True )

In [20]:
from keras.optimizers import Adam
opt = Adam(learning_rate=HP_init_lr, decay = HP_init_lr/HP_epochs)
setlabels = set(labels)
classcount = len(setlabels)
model = AnacondaVGG.buildmodel(HP_image_dim[0],HP_image_dim[1],HP_image_dim[2],classcount)

In [21]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:

history = model.fit_generator(aug.flow(trainx, trainy, batch_size=HP_batch_size),
                                       validation_data=(testx,testy),
                              steps_per_epoch= len(trainx)// HP_batch_size,
                              epochs= HP_epochs)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
